In [1]:
import os
os.environ["ZHIPUAI_API_KEY"] = '056744e429f9da058272391d1107ebe6.7aXbE4ZlIuYn7yoL'

import asyncio

from metagpt.actions import Action, UserRequirement
from metagpt.logs import logger
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.environment import Environment

from metagpt.const import MESSAGE_ROUTE_TO_ALL

2024-05-19 21:14:21.134 | INFO     | metagpt.const:get_metagpt_package_root:32 - Package root set to /root/autodl-tmp/MetaGPT
2024-05-19 21:14:21.548 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.ZHIPUAI


In [2]:
# 创建环境
classroom = Environment()

class WritePoem(Action):

    name: str = "WritePoem"

    PROMPT_TEMPLATE: str = """
    Here is the historical conversation record : {msg} .
    Write a poem about the subject provided by human, Return only the content of the generated poem with NO other texts.
    If the teacher provides suggestions about the poem, revise the student's poem based on the suggestions and return.
    your poem:
    """

    async def run(self, msg: str):

        prompt = self.PROMPT_TEMPLATE.format(msg = msg)

        rsp = await self._aask(prompt)

        return rsp


class ReviewPoem(Action):

    name: str = "ReviewPoem"

    PROMPT_TEMPLATE: str = """

    Here is the historical conversation record : {msg} .
    Check student-created poems about the subject provided by human and give your suggestions for revisions. You prefer poems with elegant sentences and retro style.
    Return only your comments with NO other texts.
    your comments:
    """

    async def run(self, msg: str):

        prompt = self.PROMPT_TEMPLATE.format(msg = msg)

        rsp = await self._aask(prompt)

        return rsp

In [3]:
# 接着我们定义 Student 角色与 Teacher 角色，与单智能体不同的部分是，我们需要声明每个角色关注的动作（self._watch），只有当关注的动作发生后，角色才会开始行动

class Student(Role):

    name: str = "xiaoming"
    profile: str = "Student"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._init_actions([WritePoem])
        self._watch([UserRequirement, ReviewPoem])  # UserRequirement是人类指令的意思

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: ready to {self.rc.todo}")
        todo = self.rc.todo

        msg = self.get_memories()  # 获取所有记忆
        # logger.info(msg)
        poem_text = await WritePoem().run(msg)
        logger.info(f'student : {poem_text}')
        msg = Message(content=poem_text, role=self.profile,
                      cause_by=type(todo))

        return msg

class Teacher(Role):

    name: str = "laowang"
    profile: str = "Teacher"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._init_actions([ReviewPoem])
        self._watch([WritePoem])

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: ready to {self.rc.todo}")
        todo = self.rc.todo

        msg = self.get_memories()  # 获取所有记忆
        poem_text = await ReviewPoem().run(msg)
        logger.info(f'teacher : {poem_text}')
        msg = Message(content=poem_text, role=self.profile,
                      cause_by=type(todo))

        return msg

In [4]:
async def main(topic: str, n_round=3):

    classroom.add_roles([Student(), Teacher()])

    classroom.publish_message(
        Message(role="Human", content=topic, cause_by=UserRequirement,
                send_to='' or MESSAGE_ROUTE_TO_ALL),
        peekable=False,
    )

    while n_round > 0:
        # self._save()
        n_round -= 1
        logger.debug(f"max {n_round=} left.")

        await classroom.run()
    return classroom.history

await main(topic='wirte a poem about moon')

2024-05-19 21:27:20.806 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.ZHIPUAI
2024-05-19 21:27:20.821 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.ZHIPUAI
2024-05-19 21:27:20.825 | INFO     | __main__:_act:14 - xiaoming(Student): ready to WritePoem
2024-05-19 21:27:20.826 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.ZHIPUAI


 When the moonbeams dance upon the sea,
A silver path appears to me.
In its gentle glow, I find my way,
To dream of days gone by, and yet to come.

The moon's soft light, a mystic key,
Unlocks the portal to my heart's desire.
With every full moon, a new beginning,
A chance to start anew, to love and to learn.

A celestial guide, she watches over us,
Her radiance a beacon in the night.
The moon, a constant in our ever-changing lives,
A symbol of hope and guidance given.

In the quiet hours, when darkness falls,
I look to the moon for answers true.
Her silent wisdom speaks to my soul,
Affirming the path I'm meant to pursue.

For in the moon's embrace, I find my light,
And in her reflection, my true self revealed.
She whispers secrets to the listening wind,
Empowering me to dream beyond the sky.

Oh, moon of my dreams, forever guide me,
With your divine presence, light my way.
Together we'll journey through the seasons,
Hand in hand, until the end of day.

2024-05-19 21:27:35.855 | INFO     | metagpt.utils.cost_manager:update_cost:48 - Total running cost: $0.000 | Max budget: $10.000 | Current cost: $0.000, prompt_tokens: 80, completion_tokens: 277
2024-05-19 21:27:35.857 | INFO     | __main__:_act:20 - student :  When the moonbeams dance upon the sea,
A silver path appears to me.
In its gentle glow, I find my way,
To dream of days gone by, and yet to come.

The moon's soft light, a mystic key,
Unlocks the portal to my heart's desire.
With every full moon, a new beginning,
A chance to start anew, to love and to learn.

A celestial guide, she watches over us,
Her radiance a beacon in the night.
The moon, a constant in our ever-changing lives,
A symbol of hope and guidance given.

In the quiet hours, when darkness falls,
I look to the moon for answers true.
Her silent wisdom speaks to my soul,
Affirming the path I'm meant to pursue.

For in the moon's embrace, I find my light,
And in her reflection, my true self revealed.
She whispers secr

 1. Beautiful imagery and sentiment expressed in the opening lines. Consider incorporating more intricate language to add a touch of elegance and a retro feel, such as "When moonbeams twirl upon the cerulean sea, a silvery path materializes before my eyes."

2. The poem's structure is well-organized, but you may want to vary the sentence structure to create a more fluid flow. Consider using more complex sentences in some lines to balance the simpler ones.

3. The phrase "a new beginning" could be replaced with a more distinctive and poetic expression, such as "a rebirth of possibilities," to add more depth to the line.

4. To emphasize the retro feel, you might want to use more romantic and descriptive words, such as "heavenly guide" or "shining beacon" instead of "celestial guide" and "radiance."

5. The poem would benefit from a few lines that describe the moon's beauty and allure more explicitly. This could help create a stronger connection between the moon and the speaker's emotion

2024-05-19 21:27:57.112 | INFO     | metagpt.utils.cost_manager:update_cost:48 - Total running cost: $0.000 | Max budget: $10.000 | Current cost: $0.000, prompt_tokens: 352, completion_tokens: 401
2024-05-19 21:27:57.114 | INFO     | __main__:_act:42 - teacher :  1. Beautiful imagery and sentiment expressed in the opening lines. Consider incorporating more intricate language to add a touch of elegance and a retro feel, such as "When moonbeams twirl upon the cerulean sea, a silvery path materializes before my eyes."

2. The poem's structure is well-organized, but you may want to vary the sentence structure to create a more fluid flow. Consider using more complex sentences in some lines to balance the simpler ones.

3. The phrase "a new beginning" could be replaced with a more distinctive and poetic expression, such as "a rebirth of possibilities," to add more depth to the line.

4. To emphasize the retro feel, you might want to use more romantic and descriptive words, such as "heavenly 

 When moonbeams twirl upon the cerulean sea,
A silvery path materializes before my eyes.
In their gentle glow, I find my way to dream,
Of days gone by and yet to come, oh moon of my dreams.

Thy radiant beams, a mystic key unlock,
The portal to my heart's desire, my celestial guide.
With every full moon, a rebirth of possibilities,
A chance to start anew, to love and to learn, oh moonlit night.

Oh, shining beacon in the night, thy light divine,
A constant in our ever-changing lives, a symbol of hope and guidance given.
In the quiet hours, when darkness falls, I look to thee,
For answers true, thy silent wisdom speaks to my soul.

Thy embrace, a haven, reveals my true self,
In thy reflection, I find my light, my guide eternal.
Thou dost whisper secrets to the listening wind,
Empowering me to dream beyond the sky, oh lunar muse.

Oh, moon of my dreams, forever guide me,
With thy divine presence, light my way through life's journey.
Together we'll journey through the seasons, hand in han

2024-05-19 21:28:12.864 | INFO     | metagpt.utils.cost_manager:update_cost:48 - Total running cost: $0.001 | Max budget: $10.000 | Current cost: $0.000, prompt_tokens: 763, completion_tokens: 291
2024-05-19 21:28:12.866 | INFO     | __main__:_act:20 - student :  When moonbeams twirl upon the cerulean sea,
A silvery path materializes before my eyes.
In their gentle glow, I find my way to dream,
Of days gone by and yet to come, oh moon of my dreams.

Thy radiant beams, a mystic key unlock,
The portal to my heart's desire, my celestial guide.
With every full moon, a rebirth of possibilities,
A chance to start anew, to love and to learn, oh moonlit night.

Oh, shining beacon in the night, thy light divine,
A constant in our ever-changing lives, a symbol of hope and guidance given.
In the quiet hours, when darkness falls, I look to thee,
For answers true, thy silent wisdom speaks to my soul.

Thy embrace, a haven, reveals my true self,
In thy reflection, I find my light, my guide eternal.


 gaze.

'\nHuman: wirte a poem about moon\nStudent:  When the moonbeams dance upon the sea,\nA silver path appears to me.\nIn its gentle glow, I find my way,\nTo dream of days gone by, and yet to come.\n\nThe moon\'s soft light, a mystic key,\nUnlocks the portal to my heart\'s desire.\nWith every full moon, a new beginning,\nA chance to start anew, to love and to learn.\n\nA celestial guide, she watches over us,\nHer radiance a beacon in the night.\nThe moon, a constant in our ever-changing lives,\nA symbol of hope and guidance given.\n\nIn the quiet hours, when darkness falls,\nI look to the moon for answers true.\nHer silent wisdom speaks to my soul,\nAffirming the path I\'m meant to pursue.\n\nFor in the moon\'s embrace, I find my light,\nAnd in her reflection, my true self revealed.\nShe whispers secrets to the listening wind,\nEmpowering me to dream beyond the sky.\n\nOh, moon of my dreams, forever guide me,\nWith your divine presence, light my way.\nTogether we\'ll journey through the se

# Team

In [5]:
class Team(BaseModel):
    """
    Team: Possesses one or more roles (agents), SOP (Standard Operating Procedures), and a env for instant messaging,
    dedicated to env any multi-agent activity, such as collaboratively writing executable code.
    """

    model_config = ConfigDict(arbitrary_types_allowed=True)

    env: Environment = Field(default_factory=Environment)
    # investment 用于管理团队成本（即限制token花费），idea效果则等同于告诉你的团队接下来该围绕什么工作
    investment: float = Field(default=10.0)
    idea: str = Field(default="")

    def hire(self, roles: list[Role]):
        """Hire roles to cooperate"""
        self.env.add_roles(roles)

    def invest(self, investment: float):
        """Invest company. raise NoMoneyException when exceed max_budget."""
        self.investment = investment
        CONFIG.max_budget = investment
        logger.info(f"Investment: ${investment}.")

    def run_project(self, idea, send_to: str = ""):
            """Run a project from publishing user requirement."""
            self.idea = idea
    
            # Human requirement.
            self.env.publish_message(
                Message(role="Human", content=idea, cause_by=UserRequirement, send_to=send_to or MESSAGE_ROUTE_TO_ALL),
                peekable=False,
            )

    # 在 Team 运行时，首先将调用 run_project 方法给智能体们一个需求，接着在 n_round 的循环中，重复检查预算与运行 env，最后返回环境中角色的历史对话
    @serialize_decorator
    async def run(self, n_round=3, idea="", send_to="", auto_archive=True):
        """Run company until target round or no money"""
        if idea:
            self.run_project(idea=idea, send_to=send_to)

        while n_round > 0:
            # self._save()
            n_round -= 1
            logger.debug(f"max {n_round=} left.")
            self._check_balance()

            await self.env.run()
        self.env.archive(auto_archive)
        return self.env.history

NameError: name 'BaseModel' is not defined

In [9]:
"""
Filename: MetaGPT/examples/build_customized_multi_agents.py
Created Date: Wednesday, November 15th 2023, 7:12:39 pm
Author: garylin2099
"""
import re

import fire

from metagpt.actions import Action, UserRequirement
from metagpt.logs import logger
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.team import Team


def parse_code(rsp):
    pattern = r"```python(.*)```"
    match = re.search(pattern, rsp, re.DOTALL)
    code_text = match.group(1) if match else rsp
    return code_text


class SimpleWriteCode(Action):
    PROMPT_TEMPLATE: str = """
    Write a python function that can {instruction}.
    Return ```python your_code_here ``` with NO other texts,
    your code:
    """
    name: str = "SimpleWriteCode"

    async def run(self, instruction: str):
        prompt = self.PROMPT_TEMPLATE.format(instruction=instruction)

        rsp = await self._aask(prompt)

        code_text = parse_code(rsp)

        return code_text


class SimpleCoder(Role):
    name: str = "Alice"
    profile: str = "SimpleCoder"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._watch([UserRequirement])
        self.set_actions([SimpleWriteCode])


class SimpleWriteTest(Action):
    PROMPT_TEMPLATE: str = """
    Context: {context}
    Write {k} unit tests using pytest for the given function, assuming you have imported it.
    Return ```python your_code_here ``` with NO other texts,
    your code:
    """

    name: str = "SimpleWriteTest"

    async def run(self, context: str, k: int = 3):
        prompt = self.PROMPT_TEMPLATE.format(context=context, k=k)

        rsp = await self._aask(prompt)

        code_text = parse_code(rsp)

        return code_text


class SimpleTester(Role):
    name: str = "Bob"
    profile: str = "SimpleTester"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([SimpleWriteTest])
        # self._watch([SimpleWriteCode])
        self._watch([SimpleWriteCode, SimpleWriteReview])  # feel free to try this too

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: to do {self.rc.todo}({self.rc.todo.name})")
        todo = self.rc.todo

        # context = self.get_memories(k=1)[0].content # use the most recent memory as context
        context = self.get_memories()  # use all memories as context

        code_text = await todo.run(context, k=5)  # specify arguments
        msg = Message(content=code_text, role=self.profile, cause_by=type(todo))

        return msg


class SimpleWriteReview(Action):
    PROMPT_TEMPLATE: str = """
    Context: {context}
    Review the test cases and provide one critical comments:
    """

    name: str = "SimpleWriteReview"

    async def run(self, context: str):
        prompt = self.PROMPT_TEMPLATE.format(context=context)

        rsp = await self._aask(prompt)

        return rsp


class SimpleReviewer(Role):
    name: str = "Charlie"
    profile: str = "SimpleReviewer"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([SimpleWriteReview])
        self._watch([SimpleWriteTest])


async def main(
    idea: str = "write a function that calculates the product of a list",
    investment: float = 3.0,
    n_round: int = 5,
    add_human: bool = False,
):
    logger.info(idea)

    team = Team()
    team.hire(
        [
            SimpleCoder(),
            SimpleTester(),
            SimpleReviewer(is_human=add_human),
        ]
    )

    team.invest(investment=investment)
    team.run_project(idea)
    await team.run(n_round=n_round)


if __name__ == "__main__":
    fire.Fire(main)

2024-05-19 21:55:54.922 | INFO     | __main__:main:122 - write a function that calculates the product of a list
2024-05-19 21:55:54.925 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.ZHIPUAI


AttributeError: 'SimpleCoder' object has no attribute 'set_actions'